# Funktory

Funktor, inaczej __obiekt funkcyjny__, to każdy obiekt, którego można użyć jak funkcji. <br>
&emsp;- wskaźniki do funkcji, <br>
&emsp;- obiekt klasy z przeładowanym operatorem (), <br>
&emsp;- wyrażenia lambda.

## 1. Wskaźnik do funkcji

In [1]:
#include <iostream>
#include <string>

int comp_int(const void * num1, const void * num2) 
{
    return *(const int*)num1 - *(const int *)num2;
}

In [2]:
int (*comp)(const void*, const void *) = comp_int; // pointer to function

int arr[] = {37,4,56,7,0,0,21,-1};
const int size = sizeof(arr)/sizeof(int);

qsort(arr, size, sizeof(int), comp); // using functor
for (auto num : arr) std::cout << num << " ";
std::cout << std::endl;

-1 0 0 4 7 21 37 56 


## 2. Obiekt klasy z przeładowanym operatorem ()

__Właściwości__ <br>
&emsp;- można zdefiniować blisko jego użycia (np. wewnątrz innej funkcji lub innej klasy) <br>
&emsp;- posiada stan

In [3]:
/// Converts given unsigned to binary or hexadecimal
class Convert
{
public:
    /// ctor sets default type converting to to_bin
    Convert() : _conv{&Convert::to_bin} {}
    ~Convert() {};
    
    
    // switching functions
    
    /// Switches converter to 'to binary' mode
    void bin()
    {
        _conv = &Convert::to_bin;
    }
    /// Switches converter to 'to hexadecimal' mode
    void hex()
    {
        _conv = &Convert::to_hex;
    }
    
    
    /// Overloaded operator() allows us to use object as function
    /// @param num Number to convert
    /// @returns String representing converted number
    std::string operator()(const unsigned num) 
    {
        return (this->*_conv)(num);
    }

private:
    /// Pointer to function that actually does converting
    std::string (Convert::*_conv)(const unsigned); 
    
    // converting functions
    std::string to_bin(const unsigned num)
    {
        std::string str{""};
        for (unsigned temp = num ; temp != 0 ; temp /= 2)
            str.insert(str.begin(), ((temp%2 == 1) ? '1' : '0'));
        return str;
    }
    
    std::string to_hex(const unsigned num)
    {
        std::string str{""};
        for (unsigned temp = num ; temp != 0 ; temp /= 16)
            str.insert(str.begin(), ((temp%16<10) ? static_cast<char>((temp%16)+48) : static_cast<char>((temp%16)+55)));
        return str;
    }
    
};

In [4]:
Convert convert;
const unsigned num = 51966;
// dec to bin
std::cout << "Using default state, decimal to binary.\n\t";
std::cout << convert(num) <<std::endl;

// dec to hex
convert.hex();
std::cout << "Switching to hex mode.\n\t";
std::cout << convert(num) <<std::endl;

// dec to bin
convert.bin();
std::cout << "Back to bin mode.\n\t";
std::cout << convert(num) <<std::endl;

Using default state, decimal to binary.
	1100101011111110
Switching to hex mode.
	CAFE
Back to bin mode.
	1100101011111110


## 3. Wyrażenia lambda (funkcja anonimowa)

__Właściwości__<br>
&emsp;- można zdefiniować bardzo blisko jego użycia<br>
&emsp;- przechwytywanie<br>
&emsp;- posiada stan

__Sposób użycia:__<br>
```[ captures ] ( params ) lambda-specifiers -> T { body }```

\[\] - początek wyrażenia lambda, przechwytywanie nazw<br>
() - między okrągłe nawiasy podajemy argumenty jak w zwykłej funkcji, opcjonalne<br>
atrybuty wyrażenia lambda np. ```mutable```, opcjonalne<br>
-> T - zwracany typ, opcjonalne<br>
{} - analogicznie jak w zwykłych funkcjach, ciało funkcji

### Przykład

In [5]:
// #include <vector> // std::vector
#include <numeric> // std::iota
#include <algorithm> // std::transform

std::vector<int> foo(10);
std::vector<int> bar(10);

std::iota(foo.begin(), foo.end(), 0); // fill vec with 0, 1, 2, ...

for (auto num : foo)
    std::cout << num << " ";
std::cout << std::endl;

// lambda returns every element of foo raised to the power of 2
std::transform(foo.begin(), foo.end(), bar.begin(), [](int x){return x*x; });

for (auto num : bar)
    std::cout << num << " ";
std::cout << std::endl;

0 1 2 3 4 5 6 7 8 9 
0 1 4 9 16 25 36 49 64 81 


### Przechwytywanie nazw

Zmienne automatyczne można przechwytywać na dwa sposoby, przez __wartość__, lub __referencję__. W pierwszym przypadku taka zmienna będzie (*prawie* zawsze) __stała__.

In [6]:
int x = 10, y = 3;
std::cout <<[ x, & y ] { return x + y; }();

13

Zmienna ```x``` jest przechwycona przez wartość, a ```y``` przez referencję.

```[=]``` - wszystkie zmienne będą przechwytywane przez wartość,<br>
```[&]``` - wszystkie zmienne będą przechwytywane przez referencję,<br>
```[=, &x]``` - wszystkie zmienne będą przechwytywane przez wartość, ale ```x``` przez referencję,<br>
```[&, y, z]``` - wszystkie zmienne będą przechwytywane przez referencję, ale ```y``` i ```z``` przez wartość.

### Atrybuty

Zmienne przechwytywane przez wartość są stałe. Można to zmienić używając atrybutu ```mutable```.

In [7]:
int a = 5;
std::cout << [a]{a=10; return a;}(); // error

input_line_16:3:19: error: cannot assign to a variable captured by copy in a non-mutable lambda
std::cout << [a]{a=10; return a;}(); // error
                 ~^


Interpreter Error: 

In [8]:
int a = 5;
std::cout << [a]() mutable {a=10; return a;}(); 

10

### Typ

Typ wyrażenia lambda jest oficjalnie nieokreślony.

In [9]:
auto lam = [](int x){return ++x;};
int x = 0;
std::cout << lam(x);

1

Lambda nie jest funkcją, jest funktorem i posiada stan. 

In [10]:
int i = 0;
int j = 0;
auto lambda = [i, &j]()mutable
{
    ++i;
    ++j;
    std::cout << "Inside lambda: \ti = " << i << ", j = " << j << std::endl;
};
lambda();
lambda();
std::cout << "In main: \ti = " << i << ", j = " << j << std::endl;

input_line_19:4:1: warning: captures will be by reference, not copy
auto lambda = [i, &j]()mutable
^


Inside lambda: 	i = 1, j = 1
Inside lambda: 	i = 2, j = 2
In main: 	i = 0, j = 2


In [11]:
int i = 0;
int j = 0;
auto lambda = [&, i]()mutable
{
    ++i;
    ++j;
    std::cout << "Inside lambda: \ti = " << i << ", j = " << j << std::endl;
};
lambda();
i = 5;
j = 5;
std::cout << "In main: \ti = " << i << ", j = " << j << std::endl;
lambda();
std::cout << "In main: \ti = " << i << ", j = " << j << std::endl;

input_line_20:4:1: warning: captures will be by reference, not copy
auto lambda = [&, i]()mutable
^


Inside lambda: 	i = 1, j = 1
In main: 	i = 5, j = 5
Inside lambda: 	i = 2, j = 6
In main: 	i = 5, j = 6


Od C++14, zmienną w stanie lambdy można rówież utworzyć (należy użyć atrybutu ```mutable```).

In [12]:
auto lambda = [ how_many = 0]() mutable 
{
    std::cout << "Lambda called: " << ++how_many << " time(s)" << std::endl;
};
lambda();
lambda();

input_line_21:2:2: warning: captures will be by reference, not copy
 auto lambda = [ how_many = 0]() mutable 
 ^


Lambda called: 1 time(s)
Lambda called: 2 time(s)


### Przekazywanie do funkcji

Lambdę można przekazać do funkcji w miejsce, w którym oczekuje ona wskaźnika do funkcji (pod warunkiem że typ się zgadza).

In [13]:
int g(int x, int (*f)(int))
{
    return 2*f(x);
}

std::cout << g(2, [](int x){return 2*x;}) << std::endl;

8


In [14]:
int g(int x, int (*f)(int))
{
    return 2*f(x);
}

std::cout << g(2, [](int x){return 2.*x;}) << std::endl; // in lambda: double instead of int, error!!

input_line_23:6:15: error: no matching function for call to 'g'
 std::cout << g(2, [](int x){return 2.*x;}) << std::endl; // in lambda: double instead of int, error!!
              ^
input_line_23:1:5: note: candidate function not viable: no known conversion from '(lambda at input_line_23:6:20)' to 'int (*)(int)' for 2nd argument
int g(int x, int (*f)(int))
    ^


Interpreter Error: 

In [15]:
int g(int x, int (*f)(int))
{
    return 2*f(x);
}

std::cout << g(2, [](int x) -> int {return 2.*x;}) << std::endl;

8


Co gdy wysłaną do funkcji lambdą chcemy wykonać przechwytywanie?

In [16]:
int g(int x, int (*f)(int))
{
    return 2*f(x);
}

int y = 50;
std::cout << g(2, [&](double x){return y*x;}) << std::endl; // error!!

input_line_25:7:14: error: no matching function for call to 'g'
std::cout << g(2, [&](double x){return y*x;}) << std::endl; // error!!
             ^
input_line_25:1:5: note: candidate function not viable: no known conversion from '(lambda at input_line_25:7:19)' to 'int (*)(int)' for 2nd argument
int g(int x, int (*f)(int))
    ^


Interpreter Error: 

In [17]:
#include <functional>
int g(int x, std::function< int(int) > f)
{
    return 2*f(x);
}

int y = 50;
std::cout << g(2, [&](double x){return y*x;}) << std::endl;

200


Do ```std::function``` można również przypisać obiekt klasy z przeładowanym operatorem ().

In [18]:
struct PrintInt
{
    void operator()(int x)
    {
        std::cout << x << std::endl;
    }
};

PrintInt printInt;
std::function<void(int)> print_int = printInt;
print_int(7);

7


## Kilka pojęć związanych z funktorami

1. __Generator__ - funktor, który można wywołać bez żadnych argumentów

In [19]:
std::vector<int> v(10);
std::generate(v.begin(), v.end(), [i = 1]()mutable{return ++i;});
for (auto n : v)
    std::cout << n << " ";

2 3 4 5 6 7 8 9 10 11 

2. __Predykat__ - funktor zwracający wartość logiczną

In [20]:
std::cout << std::count_if(v.begin(), v.end(), [](int x){return x%2==0;}) << std::endl;

5


3. __Funktory predefiniowane__ - w bibliotece STL znajduje się kilka już zdefiniowanych funktorów, które mają na celu ułatwić pracę z funkcjami tej biblioteki.

In [21]:
std::negate<int> neg; // creating object of predefined negate functor
std::vector<int> v2(10);
std::transform(v.begin(), v.end(), v2.begin(), neg); 
for (auto n : v2)
    std::cout << n << " ";

-2 -3 -4 -5 -6 -7 -8 -9 -10 -11 

## Koniec